In [2]:
#Code for widgets from https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Layout.html
#This program obtains a list of assignments from a csv file and reproduces the information in a go-table format.
#It also has two interactive widgets from ipywidgets that are used to mark one or more assignments as complete.

import plotly.graph_objects as go

import pandas as pd

import ipywidgets as widgets


#Read csv data and data to a new column titled 'Colors'
df = pd.read_csv('testdata.csv')
df['Colors'] = 'nil'
for index,row in df.iterrows():
    if row['Status'] == 'Open':
        df.at[index,'Colors'] = 'black'
    else:
        df.at[index,'Colors'] = 'red'

#Create a list of Assignments to be used a list source for the drop down list widget
a_list = list(df['ASSIGNMENT NAME'])
a_list.insert(0,'None')

#Pressing the button marks an assginment as complete
button = widgets.Button(description="Mark As Complete")
text = widgets.Dropdown(
    options=a_list,
    value=a_list[0],
    description='Assignment Name:',
    disabled=False,
)

#define the outputs
output = widgets.Output()

#Define actions to be performed on button click
#@output.capture()
def on_button_clicked(b):
    with output:
        output.clear_output()
        a_name = text.value #obtain value of the drop down list
        
        if a_name != 'None': #if value is 'None', no action is taken
            
            # identify the relevant assignment from database.
            #It is assumed here that there are no duplicate assignment names
            index_req = df.index[df['ASSIGNMENT NAME'] == a_name].to_list() 
            
            #Mark the identified assignment as complete and sort the table based on Status followed by Date.
            df.at[index_req[0],'Status'] = 'Closed'
            df.at[index_req[0],'Colors'] = 'red'
            df.sort_values(by=['Status', 'DATE'], inplace=True, ascending=[False,True])
            
            #update drop down list
            a_list = list(df['ASSIGNMENT NAME'])
        
        #write the updated database to a csv file
        df.to_csv('testdata.csv',columns=['DATE','WEEK NUMBER','COURSE','ASSIGNMENT NAME','DESCRIPTION','Status'])
        
        #define the go-table
        fig = go.Figure(data=[go.Table(header=dict(
        values=["Date", "Week Number","Course","Assignment Name","Description","Status"],
        line_color='white', fill_color='white',
        align='center', font=dict(color='black', size=12)
        ),
        cells=dict(
        values=[df['DATE'], df['WEEK NUMBER'],df['COURSE'],df['ASSIGNMENT NAME'],df['DESCRIPTION'],df['Status']],
        #line_color=[df.Colors], 
        #fill_color=[df.Colors],
        align='center', font=dict(color= [df.Colors], size=11)
         ))
        ])
        
        #show the go-table
        fig.show()
        
#fig = go.Figure(data=[go.Table(header=dict(
#        values=["Date", "Week Number","Course","Assignment Name","Description","Status"],
#        line_color='white', fill_color='white',
#        align='center', font=dict(color='black', size=12)
#          ),
#        cells=dict(
#        values=[df['DATE'], df['WEEK NUMBER'],df['COURSE'],df['ASSIGNMENT NAME'],df['DESCRIPTION'],df['Status']],
#        #line_color=[df.Colors], 
#        #fill_color=[df.Colors],
#        align='center', font=dict(color= [df.Colors], size=11)
#          ))
#        ])



button.on_click(on_button_clicked)
list_button = widgets.HBox([text,button])
widgets.VBox([list_button,output])


